# Import

In [1]:
import pandas as pd
import numpy as np
import random

# Function

In [2]:
### Get points
def get_points(data='att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()
#     t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone
    ### Denominator
    tsa = [[t0[i][j]**alpha for i in range(all_points)] for j in range(all_points)] # t square alpha
    nsb = [[n[i][j]**beta for i in range(all_points)] for j in range(all_points)] # n square beta
    txn = [[tsa[i][j] * nsb[i][j] for i in range(all_points)] for j in range(all_points)] # tsa * nsb
    denominator = sum(sum(np.array(txn))) # sum(np.array(txn) : Sum of fist row, # sum(sum(np.array(txn)) : Sum of all rows

### Swap paths position
def swapPositions(list, pos1, pos2):
    list_swap_split = []
    for i in range(len(list_split_paths)):
        swap_split_paths = []
        for j in list_split_paths[i]:
            j[0], j[1] = j[1], j[0]
            swap_split_paths.append(j)
        list_swap_split.append(swap_split_paths)
    return list_swap_split

In [3]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 1
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone
### Denominator
tsa = [[t0[i][j]**alpha for i in range(all_points)] for j in range(all_points)] # t square alpha
nsb = [[n[i][j]**beta for i in range(all_points)] for j in range(all_points)] # n square beta
txn = [[tsa[i][j] * nsb[i][j] for i in range(all_points)] for j in range(all_points)] # tsa * nsb
denominator = sum(sum(np.array(txn))) # sum(np.array(txn) : Sum of fist row, # sum(sum(np.array(txn)) : Sum of all rows

list_gens = []
list_all_paths = []
list_all_total_distance = []

list_pheromones = {}

for gen in range(gens):
    print(f'Gen : {gen}')

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])
        print(f'Total Distance : {total_distance}')

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        print(paths)


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # lpwostp = list_paths.copy()
    # lpwostp.pop(shortest_distance_index) # List paths without shortest path

#     for s in range(len(shortest_path)):
#         for i in range(len(cm)):
#             for j in range(len(cm)):
#                 try:
#                     if i==shortest_path[s] and j==shortest_path[s+1]:
#                         t_new = ((1-p)*t0[shortest_path[s]][shortest_path[s+1]]) + (100/shortest_distance_value)
#                         t0[shortest_path[s]][shortest_path[s+1]] = t_new
#                     else:
#                         t_new = ((1-p)*t0[i][j])
#                         t0[i][j] = t_new
# #                     else:
# #                         t0[i][j] = 0
#                 except:
#                     pass

    # list_pheromones.update({gen : t0})

    print(list_paths)

#     print(f't0 of ant {gen+1} : {t0}')
    # print(f'List total distance : {list_total_distance}')
    # print(f'List total path : {list_paths}')
    print(f'Shortest distance value: {shortest_distance_value}')
    print(f'Shortest distance index: {shortest_distance_index}')
    print(f'Shortest path: {shortest_path}')
    print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

    ### Pheromone reset
    # reset_values()

# print(list_pheromones)
# return [list_gens, list_ants, list_total_distance, list_paths]
    

Gen : 0
Ant : 0
Total Distance : 42268.12149826158
[3, 25, 9, 23, 41, 47, 4, 28, 1, 33, 13, 24, 12, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 38, 31, 2, 21, 15, 0, 8, 37, 30, 39, 7, 40, 34, 44]
Ant : 1
Total Distance : 36482.072968170774
[6, 17, 35, 27, 5, 36, 18, 26, 42, 29, 16, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]
Ant : 2
Total Distance : 42538.9810309781
[25, 3, 9, 23, 41, 47, 4, 28, 1, 33, 13, 24, 12, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 38, 31, 2, 21, 15, 0, 8, 37, 30, 39, 7, 40, 34, 44]
Ant : 3
Total Distance : 36921.659400007324
[30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 45, 32, 14, 11, 10, 22, 13, 24, 12, 20, 46, 19, 39, 8, 0, 7, 37, 21, 15, 2, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]
Ant : 4
Total Distance : 38703.24564638667
[20, 46, 10, 22, 13, 24, 12, 38, 31, 47, 4, 

In [71]:
list_pair_paths = []
for i in range(len(list_paths)):
    pair_paths = []
    for j in range(len(list_paths)):
        try:
            pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
        except:
            pass
    list_pair_paths.append(pair_paths)

# t = []
for i, distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
    for j in list_pair_paths[i]:
        print(j, distance)
# print(len(t))

[3, 25] 42268.12149826158
[25, 9] 42268.12149826158
[9, 23] 42268.12149826158
[23, 41] 42268.12149826158
[41, 47] 42268.12149826158
[47, 4] 42268.12149826158
[4, 28] 42268.12149826158
[28, 1] 42268.12149826158
[1, 33] 42268.12149826158
[33, 13] 42268.12149826158
[13, 24] 42268.12149826158
[24, 12] 42268.12149826158
[12, 22] 42268.12149826158
[22, 10] 42268.12149826158
[10, 11] 42268.12149826158
[11, 14] 42268.12149826158
[14, 32] 42268.12149826158
[32, 45] 42268.12149826158
[45, 43] 42268.12149826158
[43, 17] 42268.12149826158
[17, 6] 42268.12149826158
[6, 27] 42268.12149826158
[27, 35] 42268.12149826158
[35, 29] 42268.12149826158
[29, 5] 42268.12149826158
[5, 36] 42268.12149826158
[36, 18] 42268.12149826158
[18, 26] 42268.12149826158
[26, 42] 42268.12149826158
[42, 16] 42268.12149826158
[16, 19] 42268.12149826158
[19, 46] 42268.12149826158
[46, 20] 42268.12149826158
[20, 38] 42268.12149826158
[38, 31] 42268.12149826158
[31, 2] 42268.12149826158
[2, 21] 42268.12149826158
[21, 15] 42268

In [66]:
list_pair_paths

[[[3, 25],
  [25, 9],
  [9, 23],
  [23, 41],
  [41, 47],
  [47, 4],
  [4, 28],
  [28, 1],
  [1, 33],
  [33, 13],
  [13, 24],
  [24, 12],
  [12, 22],
  [22, 10],
  [10, 11],
  [11, 14],
  [14, 32],
  [32, 45],
  [45, 43],
  [43, 17],
  [17, 6],
  [6, 27],
  [27, 35],
  [35, 29],
  [29, 5],
  [5, 36],
  [36, 18],
  [18, 26],
  [26, 42],
  [42, 16],
  [16, 19],
  [19, 46],
  [46, 20],
  [20, 38],
  [38, 31],
  [31, 2],
  [2, 21],
  [21, 15],
  [15, 0],
  [0, 8],
  [8, 37],
  [37, 30],
  [30, 39],
  [39, 7],
  [7, 40],
  [40, 34],
  [34, 44]],
 [[6, 17],
  [17, 35],
  [35, 27],
  [27, 5],
  [5, 36],
  [36, 18],
  [18, 26],
  [26, 42],
  [42, 29],
  [29, 16],
  [16, 43],
  [43, 30],
  [30, 37],
  [37, 8],
  [8, 0],
  [0, 7],
  [7, 21],
  [21, 15],
  [15, 2],
  [2, 22],
  [22, 10],
  [10, 11],
  [11, 14],
  [14, 32],
  [32, 45],
  [45, 39],
  [39, 19],
  [19, 46],
  [46, 20],
  [20, 12],
  [12, 24],
  [24, 13],
  [13, 33],
  [33, 40],
  [40, 28],
  [28, 4],
  [4, 47],
  [47, 38],
  [38, 31],

In [61]:
list_swap_pair_paths = []
for i in range(len(list_pair_paths)):
    swap_pair_paths = []
    for j in range(len(list_pair_paths)):
        try:
            list_pair_paths[i][j][0], list_pair_paths[i][j][1] = list_pair_paths[i][j][1], list_pair_paths[i][j][0]
            swap_pair_paths.append(list_pair_paths[i][j][0], list_pair_paths[i][j][1])
            print(list_pair_paths[i][j][0], list_pair_paths[i][j][1])
        except:
            pass
    list_swap_pair_paths.append(swap_pair_paths)



In [59]:
list_swap_pair_paths

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [15]:
list(list_total_distance.values())

[42268.12149826158,
 36482.072968170774,
 42538.9810309781,
 36921.659400007324,
 38703.24564638667,
 35713.112998296514,
 39010.45087831735,
 37907.58005530306,
 41018.91208627347,
 38857.48438859677,
 36982.62895589434,
 37656.416875409835,
 35453.627617122576,
 38099.482230928465,
 35340.486700321555,
 36447.7346856941,
 41719.430152057634,
 37058.28838230712,
 35512.194487850786,
 38243.07955450232,
 37205.4883208474,
 37239.225898352044,
 38948.8584939329,
 35339.554179276725,
 35851.504269928904,
 39785.178827951306,
 37348.973369509375,
 36825.77760946439,
 36767.91960521253,
 41836.55083825213,
 39151.49322628053,
 41858.51000915345,
 36393.11464341938,
 37565.13939363111,
 38139.81128027202,
 38355.86967514288,
 37960.19448948187,
 37376.503159109896,
 36726.88434604734,
 39726.799210855,
 37556.89783461957,
 39964.11686816355,
 35971.41317672707,
 40948.54049952079,
 35135.455097450664,
 37225.22265046211,
 39309.04802288733,
 38126.349279601476]

[21 15]
[ 2 22]
[10 11]
[14 32]
[45 43]
[17  6]
[27 35]
[29  5]
[36 18]
[26 42]
[16 19]
[46 20]
[12 24]
[13 33]
[40 28]
[ 4 47]
[38 31]
[23  9]
[41 25]
[ 3 34]
[44  1]
[39  8]
[0 7]
[37 30]


IndexError: list index out of range